In [181]:
from competitive_sudoku.sudoku import load_sudoku
from copy import deepcopy
from collections import defaultdict
from random import choice
from time import time
import os
from itertools import islice

In [182]:
def get_possible_numbers_and_empty(board) -> set:
    """
    For the current board get the possible_moves, numbers_left and empty_squares.
    Possible_moves: The coordinates of all square that are still empty.
    Numbers_left: The numbers not yet in a group for each row/column/region
    Empty_squares: The number of empty (zero) squares for each row/column/region.
    Note: read 'possible, numbers and empty'.
    @param board: The board this should be done on.
    @return: A set with the possible_moves list, the numbers_left dictionary and the empty_squares dictionary.
    """
    
    # The variables we will be adding to while looping
    possible_moves = []
    numbers_present = {"rows": [[] for i in range(board.N)], "columns": [[] for i in range(board.N)], "regions": [[] for i in range(board.N)]}
    empty_squares = [0]*(3*board.N)

    # Loop over every square
    for row in range(board.N):        
        for column in range(board.N):
            region = square2region((row,column))
            
            value = board.get(row, column)
            empty = (value == 0)
            
            if empty:
                possible_moves.append((row,column))
                
            numbers_present["rows"][row].append(value)
            empty_squares[row] += empty
            
            numbers_present["columns"][column].append(value)
            empty_squares[column+board.N] += empty
            
            numbers_present["regions"][region].append(value)
            empty_squares[region+board.N*2] += empty
    
    numbers_left = {"rows": [], "columns": [], "regions": []}
    
    # Use the numbers that are present to calculate the numbers which are left
    for group in ["rows", "columns", "regions"]:
        for i in range(len(numbers_present["rows"])): 
            numbers_left[group].append({x for x in range(1,board.N+1) if x not in set(filter((0).__ne__, numbers_present[group][i]))})
        
    return possible_moves, numbers_left, empty_squares

def solve_sudoku(board, possible_moves: list, numbers_left: dict):
    """
    Iteratively gives a solution to the given sudoku.
    First, fills in any squares where only one number is possible, then randomly guesses.
    @param possible_moves: A list containing all still open squares/possible moves.
    @param empty_squares: A dictionary containing the missing numbers for each group.
    @return: A filled board.
    """

    move_possibilities = {} # For each move keep track of the possibilities of that move
    change_made = True

    # Inspect for each square if there is only one option for that square
    # This keeps repeating until we no longer find anything to change
    while change_made == True:
        change_made = False

        for move in possible_moves[:]:
            possibilities = set(numbers_left["rows"][move[0]] & numbers_left["columns"][move[1]] & numbers_left["regions"][square2region(move)])
        
            # If this is the case we fill this move in
            if len(possibilities) == 1:
                (number,) = possibilities

                # If this try fails we have made an incorrect guess before this
                try:
                    numbers_left["rows"][move[0]].remove(number)
                    numbers_left["columns"][move[1]].remove(number)
                    numbers_left["regions"][square2region(move)].remove(number)
                except:
                    return -1

                board.put(move[0], move[1], number)
                possible_moves.remove(move)

                if move in move_possibilities: move_possibilities.pop(move)
                change_made = True

            # If this is the case, a previous guess was wrong
            elif len(possibilities) == 0:
                return -1

            else:
                move_possibilities[move] = possibilities
    
    change_made = False
    # Inspect for each square if for one of its groups it is the only square were a number can go

    # For each row, column and region gets a set of all possibilities for all squares combined
    row_possibilities = defaultdict(list)
    column_possibilities = defaultdict(list)
    region_possibilities = defaultdict(list)
    for move in move_possibilities:
        row_possibilities[move[0]].extend(move_possibilities[move])
        column_possibilities[move[1]].extend(move_possibilities[move])
        region_possibilities[square2region(move)].extend(move_possibilities[move])

    # For each open square check for each number if it is only once in the possibilities for all squares
    for move in possible_moves[:]:
        for number in move_possibilities[move]:
            if (row_possibilities[move[0]].count(number) == 1) or \
                (column_possibilities[move[1]].count(number) == 1) or \
                    (region_possibilities[square2region(move)].count(number) == 1):

                # If this try fails we have made an incorrect guess before this
                try:
                    numbers_left["rows"][move[0]].remove(number)
                    numbers_left["columns"][move[1]].remove(number)
                    numbers_left["regions"][square2region(move)].remove(number)
                except:
                    return -1
                
                board.put(move[0], move[1], number)
                possible_moves.remove(move)

                move_possibilities.pop(move)
                change_made = True

                break
    
    # If we made any changes during the last step we return back to the start
    if change_made:
        return solve_sudoku(board, possible_moves, numbers_left)
    
    # If no more squares can be filled in, pick a random square, keep making a guesses until you hit a correct one
    if board.empty in board.squares:
        move, numbers = list(move_possibilities.items())[0]
        for number in numbers:
            new_board = deepcopy(board)
            new_board.put(move[0], move[1], number)

            new_possible_moves = possible_moves[:]
            new_possible_moves.remove(move)

            new_numbers_left = deepcopy(numbers_left)
            new_numbers_left["rows"][move[0]].remove(number)
            new_numbers_left["columns"][move[1]].remove(number)
            new_numbers_left["regions"][square2region(move)].remove(number)
            result = solve_sudoku(new_board, new_possible_moves, new_numbers_left)
                
            if result != -1:
                return result

        # If no possible number worked, a previous guess was wrong 
        return -1
    
    # If the board is full, return
    return board

def square2region(square: tuple) -> int:
    """
    From the coordinates of a square return the region the square is in.
    @param square: the x and y coordinates of a square.
    """
    region_number = square[0] - square[0] % global_m
    region_number += square[1]  // global_n
    return(region_number)

def mine(board):
    global global_m, global_n, global_N
    global_m = board.m
    global_n = board.n
    global_N = global_m*global_n
        
    possible_moves, numbers_left, empty_squares = get_possible_numbers_and_empty(board)
    solved_board = solve_sudoku(board, possible_moves, numbers_left)

In [225]:
for i in range(8):
    for j in range(8):
        square = (i,j)

        region_number = square[0] - square[0] % 3
        region_number += square[1]  // 3
        region_number = square2region
        r2 = (square[0] // 3) * 3 + (square[1] // 3)

        if region_number != r2:
            print(square)

In [183]:
from itertools import product

def solve_sudoku2(size, grid):
    """ An efficient Sudoku solver using Algorithm X."""
    
    R, C = size
    N = R * C
    X = ([("rc", rc) for rc in product(range(N), range(N))] +
         [("rn", rn) for rn in product(range(N), range(1, N + 1))] +
         [("cn", cn) for cn in product(range(N), range(1, N + 1))] +
         [("bn", bn) for bn in product(range(N), range(1, N + 1))])
    Y = dict()
    for r, c, n in product(range(N), range(N), range(1, N + 1)):
        b = (r // R) * R + (c // C) # Box number
        Y[(r, c, n)] = [
            ("rc", (r, c)),
            ("rn", (r, n)),
            ("cn", (c, n)),
            ("bn", (b, n))]
    X, Y = exact_cover(X, Y)
    for i, row in enumerate(grid):
        for j, n in enumerate(row):
            if n:
                select(X, Y, (i, j, n))
    for solution in solve(X, Y, []):
        for (r, c, n) in solution:
            grid[r][c] = n
        yield grid

def exact_cover(X, Y):
    X = {j: set() for j in X}
    for i, row in Y.items():
        for j in row:
            X[j].add(i)
    return X, Y

def solve(X, Y, solution):
    if not X:
        yield list(solution)
    else:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = select(X, Y, r)
            for s in solve(X, Y, solution):
                yield s
            deselect(X, Y, r, cols)
            solution.pop()

def select(X, Y, r):
    cols = []
    for j in Y[r]:
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].remove(i)
        cols.append(X.pop(j))
    return cols

def deselect(X, Y, r, cols):
    for j in reversed(Y[r]):
        X[j] = cols.pop()
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].add(i)

if __name__ == "__main__":
    import doctest
    doctest.testmod()

In [184]:
for board in os.listdir("boards"):
    start = time()
    for i in range(10):
        mini_start = time()
        sudoku = load_sudoku(f"boards//{board}")
        mine(sudoku)
    
    first = (time()-start)
    
    start = time()
    for i in range(10):
        mini_start = time()
        sudoku = load_sudoku(f"boards//{board}")
        
        iterr = iter(sudoku.squares)
        N = sudoku.m*sudoku.n
        new_board = [list(islice(iterr, elem)) for elem in [N]*N]

        next(solve_sudoku2((sudoku.m, sudoku.n), new_board))
    
    second = (time()-start)
    
    print(board)
    print(first/second)

easy-2x2.txt
0.6029928741092636
easy-3x3.txt
1.32846312732512
empty-2x2.txt
2.8553041434028796
empty-2x3.txt
5.864324165082086
empty-3x3.txt
14.409286203435304
empty-3x4.txt
203.17602219762747
empty-4x4.txt
33.891852413151746
hard-3x3.txt
0.921850200861159
random-2x3.txt
0.8578889875059739
random-3x3.txt
0.3149143227206693
random-3x4.txt
0.26534189754216964
random-4x4.txt
0.1611177243756665


In [185]:
start = time()
for i in range(10):
    for board in os.listdir("boards"):
        mini_start = time()
        sudoku = load_sudoku(f"boards//{board}")
        mine(sudoku)

first = time()-start
print(first)

46.465035915374756


In [186]:
start = time()
for i in range(10):
    for board in os.listdir("boards"):
        mini_start = time()
        sudoku = load_sudoku(f"boards//{board}")
        
        iterr = iter(sudoku.squares)
        N = sudoku.m*sudoku.n
        new_board = [list(islice(iterr, elem)) for elem in [N]*N]

        next(solve_sudoku2((sudoku.m, sudoku.n), new_board))

second = time()-start
print(second)

1.3349902629852295


In [187]:
start = time()
for i in range(10):
    sudoku = load_sudoku(f"boards//empty-4x4.txt")

    iterr = iter(sudoku.squares)
    N = sudoku.m*sudoku.n
    new_board = [list(islice(iterr, elem)) for elem in [N]*N]

    next(solve_sudoku2((sudoku.m, sudoku.n), new_board))

print(time()-start)

0.312030553817749


In [76]:
print(first/second)
print(second/first)

2516.3134530577545
0.0003974067693294838


In [176]:
start = time()
for i in range(10):
    for board in os.listdir("boards"):
        mini_start = time()
        sudoku = load_sudoku(f"boards//{board}")
        
        iterr = iter(sudoku.squares)
        N = sudoku.m*sudoku.n
        new_board = [list(islice(iterr, elem)) for elem in [N]*N]

        next(solve_sudoku2(board))

second = time()-start
print(second)

AttributeError: 'str' object has no attribute 'm'

In [150]:
board = load_sudoku(f"boards//random-3x3.txt")

iterr = iter(board.squares)
N = board.m*board.n
new_board = [list(islice(iterr, elem)) for elem in [N]*N]

listo = []
for i, row in enumerate(new_board):
    for j, n in enumerate(row):
        if n:
            listo.append((i,j,n))

In [161]:
board = load_sudoku(f"boards//random-3x3.txt")

listo2 = []
for row, column in product(range(9), range(9)):
    number = board.get(row, column)
    if number:
        listo2.append((row, column, number))

In [310]:
start = time()
for i in range(10):
    for board in os.listdir("boards"):
        mini_start = time()
        sudoku = load_sudoku(f"boards//{board}")
        global_m = sudoku.m
        gloabal_n = sudoku.n

        next(solve_sudoku2(sudoku))

second = time()-start
print(second)

0.8863081932067871


In [360]:
start = time()
for i in range(10):
    for board in os.listdir("boards"):
        print(board)
        
        sudoku = load_sudoku(f"boards//{board}")
        global_m = sudoku.m
        gloabal_n = sudoku.n

        solve_sudoku(sudoku)
        
second = time()-start
print(second)

easy-2x2.txt


KeyError: ('renu', (0, 3))

In [351]:
sudoku = load_sudoku(f"boards//easy-3x3.txt")
global_m = sudoku.m
gloabal_n = sudoku.n

print(next(solve_sudoku2(sudoku)))

[[8, 1, 9, 7, 3, 2, 6, 5, 4], [4, 2, 3, 6, 5, 1, 8, 9, 7], [6, 7, 5, 4, 9, 8, 2, 1, 3], [3, 5, 4, 9, 8, 7, 1, 2, 6], [1, 6, 2, 3, 4, 5, 7, 8, 9], [9, 8, 7, 1, 2, 6, 4, 3, 5], [5, 3, 1, 2, 7, 4, 9, 6, 8], [2, 4, 8, 5, 6, 9, 3, 7, 1], [7, 9, 6, 8, 1, 3, 5, 4, 2]]


In [358]:
sudoku = load_sudoku(f"boards//easy-3x3.txt")
global_m = sudoku.m
global_n = sudoku.n

print(solve_sudoku(sudoku))

KeyError: (2, 5, 1)

In [355]:
sudoku = load_sudoku(f"boards//empty-2x2.txt")
global_m = sudoku.m
global_n = sudoku.n

print(solve_sudoku(sudoku))

2 2
   4   2   1   3
   1   3   2   4
   3   1   4   2
   2   4   3   1



In [361]:
from itertools import product
def solve_sudoku(board):
    """
    A Sudoku solver using Knuth's Algorithm X to solve an Exact Cover Problem.
    Credit goes out to Ali Assaf for inspiration from their version.
    https://www.cs.mcgill.ca/~aassaf9/python/algorithm_x.html 
    @param board: The board to be solved
    @return board: A solved board
    """

    # Set X for the Exact Cover Problem
    N = board.m * board.n
    X = ([("roco", roco) for roco in product(range(N), range(N))] +
         [("ronu", ronu) for ronu in product(range(N), range(1, N + 1))] +
         [("conu", conu) for conu in product(range(N), range(1, N + 1))] +
         [("renu", renu) for renu in product(range(N), range(1, N + 1))])
    
    # Subsets Y for the Exact Cover Problem
    Y = {}
    for row, column, number in product(range(N), range(N), range(1, N + 1)):
        region = square2region((row, column))
        Y[(row, column, number)] = [
            ("roco", (row, column)),
            ("ronu", (row, number)),
            ("conu", (column, number)),
            ("renu", (region, number))]
    
    # Changes X such that we can use a dictionary instead linked lists
    X = reformat_X(X, Y)
    for row, column in product(range(N), range(N)):
        number = board.get(row, column)
        if number:
            solver_select(X, Y, (row, column, number))

    # Grabs the first solution, puts in in a board and returns it
    for solution in actual_solve(X, Y, []):
        for (row, column, number) in solution:
            board.put(row, column, number)
        return board

def reformat_X(X, Y):
    X = {i: set() for i in X}
    for key, value in Y.items():
        for i in value:
            X[i].add(key)
    return X


def actual_solve(X, Y, solution):
    if not X:
        yield list(solution)
    else:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = solver_select(X, Y, r)
            for s in actual_solve(X, Y, solution):
                yield s
            solver_deselect(X, Y, r, cols)
            solution.pop()

# Selects group for Algorithm X
def solver_select(X, Y, key):
    cols = []
    for i in Y[key]:
        for j in X[i]:
            for k in Y[j]:
                if k != i:
                    X[k].remove(j)
        cols.append(X.pop(i))
    return cols

# Deselects group for Algorithm X
def solver_deselect(X, Y, key, cols):
    for i in reversed(Y[key]):
        X[i] = cols.pop()
        for j in X[i]:
            for k in Y[j]:
                if k != i:
                    X[k].add(i)

In [276]:
from itertools import product

def solve_sudoku2(board):
    """ An efficient Sudoku solver using Algorithm X."""
    
    iterr = iter(sudoku.squares)
    N = sudoku.m*sudoku.n
    grid = [list(islice(iterr, elem)) for elem in [N]*N]
    
    R, C = board.m, board.n
    N = R * C
    X = ([("rc", rc) for rc in product(range(N), range(N))] +
         [("rn", rn) for rn in product(range(N), range(1, N + 1))] +
         [("cn", cn) for cn in product(range(N), range(1, N + 1))] +
         [("bn", bn) for bn in product(range(N), range(1, N + 1))])
    Y = dict()
    for r, c, n in product(range(N), range(N), range(1, N + 1)):
        b = (r // R) * R + (c // C) # Box number
        Y[(r, c, n)] = [
            ("rc", (r, c)),
            ("rn", (r, n)),
            ("cn", (c, n)),
            ("bn", (b, n))]

    X, Y = exact_cover(X, Y)
    for i, row in enumerate(grid):
        for j, n in enumerate(row):
            if n:
                select(X, Y, (i, j, n))
    for solution in solve(X, Y, []):
        for (r, c, n) in solution:
            grid[r][c] = n
        yield grid

def exact_cover(X, Y):
    X = {j: set() for j in X}
    for i, row in Y.items():
        for j in row:
            X[j].add(i)
    return X, Y

def solve(X, Y, solution):
    if not X:
        yield list(solution)
    else:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = select(X, Y, r)
            for s in solve(X, Y, solution):
                yield s
            deselect(X, Y, r, cols)
            solution.pop()

def select(X, Y, r):
    cols = []
    for j in Y[r]:
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].remove(i)
        cols.append(X.pop(j))
    return cols

def deselect(X, Y, r, cols):
    for j in reversed(Y[r]):
        X[j] = cols.pop()
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].add(i)

if __name__ == "__main__":
    import doctest
    doctest.testmod()